In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
import pandas as pd 
merged_df = pd.read_csv("../data/merged__target_dataset_ver1.1.csv")


In [3]:
merged_df.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration,date_unregistration,sum_click,avg_score,banked_ratio,is_dropout,target
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,NaN,934.0,82.0,0.0,0,0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,-53.0,NaN,1435.0,66.4,0.0,0,0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,-92.0,12.0,281.0,0.0,0.0,1,1
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,-52.0,NaN,2158.0,76.0,0.0,0,0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,-176.0,NaN,1034.0,54.4,0.0,0,0


In [4]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   code_module           32593 non-null  object 
 1   code_presentation     32593 non-null  object 
 2   id_student            32593 non-null  int64  
 3   gender                32593 non-null  object 
 4   region                32593 non-null  object 
 5   highest_education     32593 non-null  object 
 6   imd_band              31482 non-null  object 
 7   age_band              32593 non-null  object 
 8   num_of_prev_attempts  32593 non-null  int64  
 9   studied_credits       32593 non-null  int64  
 10  disability            32593 non-null  object 
 11  date_registration     32548 non-null  float64
 12  date_unregistration   10072 non-null  float64
 13  sum_click             32593 non-null  float64
 14  avg_score             32593 non-null  float64
 15  banked_ratio       

In [5]:
merged_df.isnull().sum()

code_module                 0
code_presentation           0
id_student                  0
gender                      0
region                      0
highest_education           0
imd_band                 1111
age_band                    0
num_of_prev_attempts        0
studied_credits             0
disability                  0
date_registration          45
date_unregistration     22521
sum_click                   0
avg_score                   0
banked_ratio                0
is_dropout                  0
target                      0
dtype: int64

In [6]:
merged_df.notnull().sum()

code_module             32593
code_presentation       32593
id_student              32593
gender                  32593
region                  32593
highest_education       32593
imd_band                31482
age_band                32593
num_of_prev_attempts    32593
studied_credits         32593
disability              32593
date_registration       32548
date_unregistration     10072
sum_click               32593
avg_score               32593
banked_ratio            32593
is_dropout              32593
target                  32593
dtype: int64

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer   # 결측치값 대체.
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [8]:
merged_df["imd_band"].value_counts()

imd_band
20-30%     3654
30-40%     3539
10-20      3516
0-10%      3311
40-50%     3256
50-60%     3124
60-70%     2905
70-80%     2879
80-90%     2762
90-100%    2536
Name: count, dtype: int64

In [9]:
merged_df["banked_ratio"].value_counts()

banked_ratio
0.000000    31605
0.500000      287
0.333333       73
1.000000       61
0.250000       57
            ...  
0.777778        1
0.700000        1
0.833333        1
0.857143        1
0.875000        1
Name: count, Length: 68, dtype: int64

In [10]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   code_module           32593 non-null  object 
 1   code_presentation     32593 non-null  object 
 2   id_student            32593 non-null  int64  
 3   gender                32593 non-null  object 
 4   region                32593 non-null  object 
 5   highest_education     32593 non-null  object 
 6   imd_band              31482 non-null  object 
 7   age_band              32593 non-null  object 
 8   num_of_prev_attempts  32593 non-null  int64  
 9   studied_credits       32593 non-null  int64  
 10  disability            32593 non-null  object 
 11  date_registration     32548 non-null  float64
 12  date_unregistration   10072 non-null  float64
 13  sum_click             32593 non-null  float64
 14  avg_score             32593 non-null  float64
 15  banked_ratio       

In [11]:
merged_df['date_registration'].isnull().value_counts()

date_registration
False    32548
True        45
Name: count, dtype: int64

In [12]:
categorical_columns = ['code_module','code_presentation','id_student','gender','region','highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'disability', 'is_dropout']
numeric_columns = ['studied_credits', 'date_registration', 'date_unregistration', 'sum_click', 'avg_score', 'banked_ratio']
target = "target"  # index =17

categorical_columns_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 10,16]
numeric_columns_index = [9,11,12,13,14,15]

In [13]:
###############################
#결측치 처리
#결측치 처리 컬럼: imd_band(범주형 -> 최빈값으로 처리), date_registration(수치형 -> mean으로 처리)
# date_unregisitration(9999값으로 채우기)

# 1. 결측치 직접 처리: date_unregistration → 9999
merged_df["date_unregistration"] = merged_df["date_unregistration"].fillna(9999)

# 1. 원래 컬럼 순서 저장
original_columns = merged_df.columns.tolist()

# 2. ColumnTransformer 설정
na_transformer = ColumnTransformer([
    ("category_imputer", SimpleImputer(strategy="most_frequent"), ['imd_band']),
    ("number_imputer", SimpleImputer(strategy="mean"), ['date_registration'])
], remainder="passthrough")

# 3. 처리 대상 및 passthrough 대상 정리
processed_columns = ['imd_band', 'date_registration']
passthrough_columns = [col for col in merged_df.columns if col not in processed_columns]

# 4. fit_transform 적용
na_values_array = na_transformer.fit_transform(merged_df)

# 5. DataFrame으로 변환 + 컬럼 순서 복원
na_values_df = pd.DataFrame(na_values_array, columns=processed_columns + passthrough_columns)
na_values_df = na_values_df[original_columns]  # 순서 복원!

In [14]:
na_values_df.isnull().sum()


code_module             0
code_presentation       0
id_student              0
gender                  0
region                  0
highest_education       0
imd_band                0
age_band                0
num_of_prev_attempts    0
studied_credits         0
disability              0
date_registration       0
date_unregistration     0
sum_click               0
avg_score               0
banked_ratio            0
is_dropout              0
target                  0
dtype: int64

In [15]:
### Feature Engineering - 컬럼별로 다르게 처리.
fe_transformer = ColumnTransformer([
    ("category_ohe", OneHotEncoder(), categorical_columns), # feature의 index로 지정.   # index는 앞에 했던 배열로 적용해줘야함.
    ("number_scaler", StandardScaler(), numeric_columns)   #feature Scaling은 연속형끼리 같은 방식을 사용 (standard or MinMax 중 택1))   
])
### DataFrame이 입력일 경우 컬럼명이나 컬럼 index를 지정할 수 있다.
### ndarray가 입력일 경우 컬럼(feature) index를 지정.
fe_transformer.fit_transform(na_values_df)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 554036 stored elements and shape (32593, 28846)>

In [16]:
# 지민

In [17]:
na_values_df

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration,date_unregistration,sum_click,avg_score,banked_ratio,is_dropout,target
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,9999.0,934.0,82.0,0.0,0,0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,-53.0,9999.0,1435.0,66.4,0.0,0,0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,-92.0,12.0,281.0,0.0,0.0,1,1
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,-52.0,9999.0,2158.0,76.0,0.0,0,0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,-176.0,9999.0,1034.0,54.4,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,-4.0,9999.0,41.0,0.0,0.0,0,1
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,-23.0,9999.0,893.0,88.111111,0.0,0,0
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,-129.0,9999.0,312.0,76.666667,0.0,0,0
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,-49.0,101.0,275.0,91.5,0.0,1,1


In [18]:
X = na_values_df.drop(columns=["target"])
y = na_values_df["target"]

In [19]:
X

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration,date_unregistration,sum_click,avg_score,banked_ratio,is_dropout
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,9999.0,934.0,82.0,0.0,0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,-53.0,9999.0,1435.0,66.4,0.0,0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,-92.0,12.0,281.0,0.0,0.0,1
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,-52.0,9999.0,2158.0,76.0,0.0,0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,-176.0,9999.0,1034.0,54.4,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,-4.0,9999.0,41.0,0.0,0.0,0
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,-23.0,9999.0,893.0,88.111111,0.0,0
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,-129.0,9999.0,312.0,76.666667,0.0,0
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,-49.0,101.0,275.0,91.5,0.0,1


In [20]:
y

0        0
1        0
2        1
3        0
4        0
        ..
32588    1
32589    0
32590    0
32591    1
32592    0
Name: target, Length: 32593, dtype: object

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [22]:
for i, col in enumerate(X.columns):
    print(f"{i}: {col}")

0: code_module
1: code_presentation
2: id_student
3: gender
4: region
5: highest_education
6: imd_band
7: age_band
8: num_of_prev_attempts
9: studied_credits
10: disability
11: date_registration
12: date_unregistration
13: sum_click
14: avg_score
15: banked_ratio
16: is_dropout


In [23]:
X.columns

Index(['code_module', 'code_presentation', 'id_student', 'gender', 'region',
       'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts',
       'studied_credits', 'disability', 'date_registration',
       'date_unregistration', 'sum_click', 'avg_score', 'banked_ratio',
       'is_dropout'],
      dtype='object')

In [24]:
categorical_columns = ['code_module','code_presentation','id_student','gender','region','highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'disability','is_dropout']
numeric_columns = ['studied_credits', 'date_registration', 'date_unregistration', 'sum_click', 'avg_score', 'banked_ratio']
# target = "target"  # index =17
target = y

categorical_columns_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 16]
numeric_columns_index = [9, 11, 12, 13, 14, 15]

In [25]:
# ColumnTransformer 정의
fe_transformer = ColumnTransformer([
    ("category_ohe", OneHotEncoder(handle_unknown='ignore'), categorical_columns),
    ("number_scaler", StandardScaler(), numeric_columns)
])

In [26]:
X_processed = fe_transformer.fit_transform(X)

In [27]:
X_processed

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 554036 stored elements and shape (32593, 28846)>

In [28]:
# 1. OneHotEncoder에서 실제 컬럼 이름 가져오기
ohe = fe_transformer.named_transformers_['category_ohe']
ohe_feature_names = ohe.get_feature_names_out(categorical_columns)

# 2. 수치형 컬럼 이름
numeric_feature_names = numeric_columns

# 3. 전체 컬럼 이름 합치기
all_feature_names = list(ohe_feature_names) + numeric_feature_names

# # 4. 희소행렬을 밀집행렬로 변환 (주의: 메모리 사용량 증가)
# X_dense = X_processed.toarray()

# # 5. DataFrame으로 변환
# X_df = pd.DataFrame(X_dense, columns=all_feature_names)

# # 6. 결과 확인
# X_df.head()


In [29]:
X_sample = pd.DataFrame(X_processed[:5].toarray(), columns=all_feature_names)
X_sample

,code_module_AAA,code_module_BBB,code_module_CCC,code_module_DDD,code_module_EEE,code_module_FFF,code_module_GGG,code_presentation_2013B,code_presentation_2013J,code_presentation_2014B,...,disability_N,disability_Y,is_dropout_0,is_dropout_1,studied_credits,date_registration,date_unregistration,sum_click,avg_score,banked_ratio
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,3.901543,-1.819956,0.668717,-0.166102,0.711168,-0.159438
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,-0.481083,0.333389,0.668717,0.129896,0.213208,-0.159438
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,-0.481083,-0.458880,-1.503464,-0.551904,-1.906313,-0.159438
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,-0.481083,0.353703,0.668717,0.557055,0.519645,-0.159438
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,-0.481083,-2.165304,0.668717,-0.107021,-0.169838,-0.159438


In [30]:
print(X_processed.shape)  # 전처리 후 X의 차원
print(y.shape)            # y의 차원


(32593, 28846)
(32593,)


In [ ]:
# 밑에는 아직 덜함

In [32]:
X

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration,date_unregistration,sum_click,avg_score,banked_ratio,is_dropout
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,9999.0,934.0,82.0,0.0,0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,-53.0,9999.0,1435.0,66.4,0.0,0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,-92.0,12.0,281.0,0.0,0.0,1
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,-52.0,9999.0,2158.0,76.0,0.0,0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,-176.0,9999.0,1034.0,54.4,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,-4.0,9999.0,41.0,0.0,0.0,0
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,-23.0,9999.0,893.0,88.111111,0.0,0
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,-129.0,9999.0,312.0,76.666667,0.0,0
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,-49.0,101.0,275.0,91.5,0.0,1


In [ ]:
transformer_pipeline = Pipeline([
    ("step1", na_transformer), 
    ("step2", fe_transformer)
])

In [ ]:
transformer_pipeline.fit_transform(df)